<a href="https://colab.research.google.com/github/donghui-0126/mini-project/blob/main/shoes-project/resell_regressor/bn_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.linear_model import LinearRegression
import warnings
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

warnings.filterwarnings('ignore')

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
df = pd.read_csv('/content/drive/MyDrive/kream_data/product_data_dpp.csv')
df.head(3)

,product_id,img_path,brand,name,color1,color2,price_og,price_resell,n_scrap,+price,tokenized_color,tokenized_bn
0,28029,crawling\product_crawling\image\28029.jpg,Nike,Nike Dunk Low Retro Black,WHITE,BLACK,129000,152800,123000.0,23800,[0 0 0 0 0 0 0 0 0 0 2 4 3 5],[ 0 0 0 0 0 0 0 0 0 0 0 0 0 2 4 ...
1,12831,crawling\product_crawling\image\12831.jpg,Nike,Nike Air Force 1 '07 Low White,WHITE,WHITE,139000,136200,151000.0,-2800,[0 0 0 0 0 0 0 0 0 0 2 4 3 4],[ 0 0 0 0 0 0 0 0 0 0 0 2 4 3 4 ...
2,21935,crawling\product_crawling\image\21935.jpg,Nike,Nike Air Force 1 '07 WB Flax,FLAX,GUM,169000,173200,69000.0,4200,[ 0 0 0 0 0 0 0 0 0 0 2 82 3 51],[ 0 0 0 0 0 0 0 0 0 ...


In [46]:
df.shape

(6103, 12)

In [47]:
bn_input_data = np.array([])

for val in df['tokenized_bn'].values:
    val = [int(x) for x in val.strip('[]').split()]
    bn_input_data = np.append(bn_input_data, val)

bn_input_data = bn_input_data.reshape([6103, 21])
bn_input_data = bn_input_data.astype(int)

In [48]:
bn_input_data.shape

(6103, 21)

In [49]:
columns = [x for x in range(3000)]
index = [y for y in range(bn_input_data.shape[0])]


bn_input_df = pd.DataFrame(index=index, columns=columns)
bn_input_df = bn_input_df.fillna(0)

for index, line in enumerate(bn_input_data):
    for elem in set(line):
        if elem < 100:
            bn_input_df.iloc[index,elem] = 1
        else:
            bn_input_df.iloc[index,0] = 1

In [50]:
# 정가를 피처에 추가해준다.

bn_input_df = pd.concat([bn_input_df, df["price_og"]], axis=1)

In [62]:
bn_input_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,price_og
0,1,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,129000
1,1,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,139000
2,1,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,169000


In [51]:
bn_input_df.shape

(6103, 3001)

In [52]:
# 신발의 브랜드와 이름은 리셀가격(신발의 본래 가치??)와 관련이 클 것 같아서 target데이터를 price_resell 로 설정함. 

target_data = np.array(df['price_resell'])
print(target_data.shape)

(6103,)


In [53]:
target_data.astype(int)

array([152800, 136200, 173200, ..., 450000, 457500, 400000])

In [54]:
for n in [10,30, 50]:
    pca = PCA(n_components=n)
    bn_pca = pca.fit_transform(bn_input_df)
    bn_pca_df = pd.DataFrame(bn_pca)
    print(sum(pca.explained_variance_ratio_))

0.9999999999293424
0.9999999999692845
0.9999999999847982


In [55]:
for n in [100,300,600]:
    pca = PCA(n_components=n)
    bn_pca = pca.fit_transform(bn_input_df)
    bn_pca_df = pd.DataFrame(bn_pca)
    print(sum(pca.explained_variance_ratio_))

0.9999999999999996
0.9999999999999996
0.9999999999999996


In [56]:
# PCA의 차원을 늘리면서  K-fold를 사용해서 어떤 모델이 가장 효율적인지 살펴보겠다. 

for n in [30, 50]:
    pca = PCA(n_components=n)
    bn_pca = pca.fit_transform(bn_input_df)
    bn_pca_df = pd.DataFrame(bn_pca)

    kf = KFold(n_splits = 5)
    
    rmse_list = []
    for train_index, test_index in kf.split(target_data):
        bn_pca_df_train, bn_pca_df_test = bn_pca_df.loc[train_index], bn_pca_df.loc[test_index]
        target_data_train, target_data_test  = target_data[train_index], target_data[test_index]

        reg = LinearRegression().fit(bn_pca_df_train, target_data_train)

        pred = reg.predict(bn_pca_df_test)
        rmse = mean_squared_error(pred, target_data_test, squared=False)
        rmse_list.append(rmse)
        # print(rmse)
        
    print("PCA:", n, "rmse:", round(np.mean(rmse_list)))

PCA: 30 rmse: 308766
PCA: 50 rmse: 458043


In [61]:
# PCA의 차원을 늘리면서  K-fold를 사용해서 어떤 모델이 가장 효율적인지 살펴보겠다. 

from lightgbm import LGBMRegressor

for n in (10, 30, 50, 100):
    pca = PCA(n_components=n)
    bn_pca = pca.fit_transform(bn_input_df)
    bn_pca_df = pd.DataFrame(bn_pca)

    kf = KFold(n_splits = 5)
    
    rmse_list = []
    for train_index, test_index in kf.split(target_data):
        bn_pca_df_train, bn_pca_df_test = bn_pca_df.loc[train_index], bn_pca_df.loc[test_index]
        target_data_train, target_data_test  = target_data[train_index], target_data[test_index]
        
        lgbm_reg = LGBMRegressor().fit(bn_pca_df_train, target_data_train)

        pred = lgbm_reg.predict(bn_pca_df_test)
        rmse = mean_squared_error(pred, target_data_test, squared=False)
        rmse_list.append(rmse)
        # print(rmse)
        
    print("PCA:", n, "rmse:", round(np.mean(rmse_list)))

PCA: 10 rmse: 282859
PCA: 30 rmse: 286216
PCA: 50 rmse: 290094
PCA: 100 rmse: 286127


In [58]:
for n in [32,64]:
    pca = PCA(n_components=n)
    bn_pca = pca.fit_transform(bn_input_df)
    bn_pca_df = pd.DataFrame(bn_pca)

    kf = KFold(n_splits = 5)

    rmse_list = []
    for train_index, test_index in kf.split(target_data):
        bn_pca_df_train, bn_pca_df_test = bn_pca_df.loc[train_index], bn_pca_df.loc[test_index]
        target_data_train, target_data_test  = target_data[train_index], target_data[test_index]

        model = keras.models.Sequential([
                keras.layers.Dense(16),
                keras.layers.Dense(8),
                keras.layers.Dense(1)
        ])

        model.compile(loss="mse")
        model.fit(bn_pca_df_train,target_data_train, epochs=20, verbose=0)
        pred = model.predict(bn_pca_df_test, verbose=0)
        rmse = mean_squared_error(target_data_test,pred, squared= False)

        rmse_list.append(rmse)
    print("PCA:", n, "rmse:", round(np.mean(rmse_list)))

PCA: 32 rmse: 380003
PCA: 64 rmse: 380571


여러 가지를 실험해봤는데....

- 주성분을 30개~50개로 축소한 lgbm 모델이 적절할듯 하다.